In [2]:
import pandas as pd
import os

In [20]:
repseq_df = pd.read_csv("repseq.microcosms.tmp.tsv", sep = "\t")
accessions_df = repseq_df.loc[:,["Query", "EC", "domain"]]
accessions_df["ISEUK"] = accessions_df["domain"].str.contains("euk")

prok_annot = pd.read_csv("prok_annot_hmm.permissive.repseq.tsv", sep = "\t")
euk_annot = pd.read_csv("euk_annot_hmm.permissive.repseq.tsv", sep = "\t")
kog_labels = pd.concat([prok_annot, euk_annot])

In [50]:
ec_koginfo = pd.read_csv("/data/luojaa/kegg_stats/KOG_merged.tsv", sep = "\t")
ec_koginfo["EC_single"] = ec_koginfo["EC"].apply(lambda x: x.split("|"))
ec_koginfo = ec_koginfo.explode("EC_single")
ec_2kog = ec_koginfo[ec_koginfo.EC_single != "none" ].loc[:,["kogid", "name", "EC_single"]]
ec_2kogs = ec_2kog.groupby("EC_single")["kogid"].apply(lambda x: "|".join(set(x))).reset_index().rename(columns={"EC_single":"EC","kogid":"KOGS"})
# drop incomplete ec numbers
ec_2kogs = ec_2kogs[~ec_2kogs.EC.str.contains("-")]
ec_2kog_dict = ec_2kogs.set_index("EC").to_dict()["KOGS"]
ec_2kog_dict["3.6.4.12"] = ec_2kog_dict["5.6.2.3"]
def skip_ec(ec):
    try:
        return ec_2kog_dict[ec]
    except:
        return "None"

In [51]:
accessions_df["KOGS"] = accessions_df.EC.apply(lambda x: skip_ec(x))


In [66]:
accessions_df = accessions_df.set_index("Query")
kog_labels = kog_labels.set_index("Query")

In [68]:
accessions_df_eckog = pd.merge(accessions_df, kog_labels, left_index = True, right_index = True, how = "left")
accessions_df_eckog = accessions_df_eckog.fillna("None")

In [87]:
def top_kog(s1, s2):
    rv = []
    l1, l2 = list(s1), list(s2)
    for i in range(len(s1)):
        # to do - in future, if DL/UC 1:1 is not priority, allow l1 = None for 'discovery' of new microcosms
        if (l1[i] == "None") or (l2[i] == "None"):
            rv.append("None")
        else:
            k = "None"
            for kog in l2[i].split("|"):
                if kog in l1[i].split("|"):
                    k = kog
                    break
            rv.append(k)
    return rv
                

In [89]:
top_kogs = top_kog(accessions_df_eckog.KOGS, accessions_df_eckog.KOG)
accessions_df_eckog["KOG_microcosm"] = top_kogs
accessions_df_eckog["EC_KOG"] = accessions_df_eckog.EC + "_" + accessions_df_eckog.KOG_microcosm
accessions_df_eckog = accessions_df_eckog.reset_index()

In [102]:
accessions_df_rel

,Query,EC,domain,ISEUK,KOGS,Prob,Score,Pairwise_cov,KOG,KOG_microcosm,EC_KOG
0,2P0U_A,2.3.1.156,euk72_ep,True,None,100.0|100.0|100.0|100.0|100.0|100.0|100.0|100....,992.1|759.7|729.9|716.8|711.8|676.1|669.9|665....,0.9322033898305084|0.9322033898305084|0.934624...,K00660|K12644|K13232|K13233|K17212|K13231|K132...,None,2.3.1.156_None
1,5YPT_F,2.3.1.156,euk72_ep,True,None,100.0|100.0|100.0|100.0|100.0|100.0|100.0|100....,968.2|725.8|705.9|694.3|679.4|658.8|656.4|633....,0.902097902097902|0.8997668997668997|0.9044289...,K00660|K17212|K12644|K21384|K13232|K13231|K213...,None,2.3.1.156_None
2,AAB22421.1,3.1.26.5,euk72_ep,True,K14533|K01164|K03537|K03539|K14527|K14530|K145...,100.0|99.7,199.4|127.6,0.8899082568807339|0.8585858585858586,K02954|K02954,None,3.1.26.5_None
3,AAF05807.1,7.1.2.2,euk72_ep,True,K02117|K02145|K02133|K02111|K02112,100.0|100.0|100.0|99.9|99.9|99.8|99.6|99.4|99....,1011.9|1004.1|788.6|242.0|233.8|205.7|176.5|16...,0.7820299500831946|0.7903494176372712|0.782029...,K02111|K02132|K02111|K02412|K03224|K22506|K021...,K02111,7.1.2.2_K02111
4,AAL73350.1,1.2.1.12,euk72_ep,True,K10705|K00134,100.0|100.0|100.0|100.0|99.9,624.8|478.0|327.0|289.0|209.6,0.5675675675675675|0.5602409638554217|0.562874...,K00134|K00134|K25224|K03472|K00150,K00134,1.2.1.12_K00134
...,...,...,...,...,...,...,...,...,...,...,...
6282281,YP_654087.1,7.1.1.2,euk72_ep,True,K00332|K00337|K13380|K00339|K05580|K05575|K055...,99.9|99.8|99.7|99.0|99.0,186.6|154.2|137.3|103.3|100.5,0.8780487804878049|0.951219512195122|0.9512195...,K03880|K00330|K05574|K22158|K05574,K03880,7.1.1.2_K03880
6282282,YP_654221.1,4.1.1.39,euk72_ep,True,K01602|K01601,100.0|100.0|99.9|99.9|99.8|99.6,1644.7|804.6|245.1|229.8|203.2|179.1,0.9834710743801652|0.8677685950413223|0.855371...,K01601|K01601|K01559|K08965|K24268|K25035,K01601,4.1.1.39_K01601
6282283,YP_740422.1,7.1.1.2,euk72_ep,True,K00332|K00337|K13380|K00339|K05580|K05575|K055...,99.7|99.0,150.6|106.2,0.7545454545454545|0.7318181818181818,K00339|K03884,K00339,7.1.1.2_K00339
6282284,YP_874516.1,3.6.4.12,euk72_ep,True,K10726|K14326|K19036|K15254|K21502|K03722|K197...,100.0|100.0|99.0,656.0|299.7|136.4,0.9583333333333334|0.9210526315789472|0.534926...,K02314|K26928|K17680,K02314,3.6.4.12_K02314


In [104]:
ec_bothdom = accessions_df_eckog.groupby("EC_KOG")["ISEUK"].apply(lambda x:len(x.unique()) == 2)
ec_bothdom_list = list(ec_bothdom[ec_bothdom].index)

In [106]:
accessions_df_rel = accessions_df_eckog[accessions_df_eckog["EC_KOG"].isin(ec_bothdom_list)]

In [109]:
acc_counts = accessions_df_rel.groupby(["EC_KOG", "ISEUK"])["Query"].count().reset_index()
ecs_enough_eukmembers = set(acc_counts[(acc_counts["ISEUK"] == True) & (acc_counts["Query"] > 5)].EC_KOG)

In [110]:
acc_counts

,EC_KOG,ISEUK,Query
0,1.1.1.100_K00059,False,3443
1,1.1.1.100_K00059,True,576
2,1.1.1.100_K11610,False,817
3,1.1.1.100_K11610,True,99
4,1.1.1.100_None,False,79
...,...,...,...
13439,7.6.2.9_K02000,True,3
13440,7.6.2.9_K05847,False,311
13441,7.6.2.9_K05847,True,1
13442,7.6.2.9_None,False,3


In [78]:
accessions_outdf = accessions_df_rel.groupby(["EC", "ISEUK"])["Query"].apply(lambda x: "\n".join(x)).reset_index()
accessions_outdf = accessions_outdf[accessions_outdf["EC"].isin(ecs_enough_eukmembers)]

In [80]:
outpathroot = "/data/luojaa/eukgen/processing/microcosm_dl/"

In [81]:
accessions_outdf

,EC,ISEUK,Query
0,1.1.1.1,False,AEJ53306_1\nAKJ28696_1\nAKR51806_1\nAKT34022_1...
1,1.1.1.1,True,CEM14143.1\nEP00001P000163\nEP00001P117662\nEP...
2,1.1.1.10,False,AHM62958_1\nALS79891_1\nANU10707_1\nAPD05872_1...
3,1.1.1.10,True,EP00005P005247\nEP00008P005642\nEP00008P010001...
4,1.1.1.100,False,ABY71998_1\nAFS48457_1\nAFS49619_1\nAGY53885_1...
...,...,...,...
6803,7.6.2.5,True,EP00163P007010\nEP00163P013418\nEP00270P001738...
6804,7.6.2.7,False,AFB78298_1\nAGB78726_1\nAHF89345_1\nAHF90646_1...
6805,7.6.2.7,True,EP00163P012848\nEP00163P017165\nEP00873P030931...
6806,7.6.2.8,False,ACX82737_1\nADB37613_1\nAGB77936_1\nAGB79451_1...


In [82]:
ecs = list(accessions_outdf["EC"])
iseuks = list(accessions_outdf["ISEUK"])
uids = list(accessions_outdf["Query"])

In [12]:
# with open("/data/luojaa/eukgen/mmseqs/ran/bothdomains.ids", "w") as outfile:
#     for i in range(len(accessions_outdf)):
#         uidchunk = uids[i]
#         for uid in uidchunk.split("\n"):
#             print(uid,file=outfile)

In [ ]:
# for i in range(len(accessions_outdf)):
#     ec, iseuk, uid = ecs[i], iseuks[i], uids[i]
#     os.system(f"mkdir {outpathroot}{ec}")
#     if iseuk:
#         outpath = f"{outpathroot}{ec}/{ec}.query.acc"
#         with open(outpath, "w") as outfile:
#             print(uid, file=outfile)
#     else:
#         outpath = f"{outpathroot}{ec}/{ec}.target.acc"
#         with open(outpath, "w") as outfile:
#             print(uid, file=outfile)